In [0]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.cuda import FloatTensor, LongTensor

import numpy as np

np.random.seed(42)

# Свёрточные нейронные сети

Напомню, свертки - это то, с чего начался хайп нейронных сетей в районе 2012-ого.

Работают они примерно так:  
![Conv example](http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif)   
From [Feature extraction using convolution](http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution).

Формально - учатся наборы фильтров, каждый из которых скалярно умножается на элементы матрицы признаков. На картинке выше исходная матрица сворачивается с фильтром
$$
 \begin{pmatrix}
  1 & 0 & 1 \\
  0 & 1 & 0 \\
  1 & 0 & 1
 \end{pmatrix}
$$

Но нужно не забывать, что свертки обычно имеют ещё такую размерность, как число каналов. Например, картинки имеют обычно три канала: RGB.  
Наглядно демонстрируется как выглядят при этом фильтры [здесь](http://cs231n.github.io/convolutional-networks/#conv).

После сверток обычно следуют pooling-слои. Они помогают уменьшить размерность тензора, с которым приходится работать. Самым частым является max-pooling:  
![maxpooling](http://cs231n.github.io/assets/cnn/maxpool.jpeg =x300)  
From [CS231n Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/#pool)

## Character-Level Convolutions
Мы говорим про свертки для работы с текстами. Совсем не очевидно, что они вообще должны помочь в работе с текстами. То есть в изображениях они отлавливают некоторые локальные особенности, такие как:
![weights](https://i.stack.imgur.com/Hl2H6.png)

Для текстов свертки работают как n-граммные детекторы (примерно). Посмотрите на пример символьной сверточной сети:

![text-convs](https://image.ibb.co/bC3Xun/2018_03_27_01_24_39.png =x500)  
From [Character-Aware Neural Language Models](https://arxiv.org/abs/1508.06615)

*Сколько учится фильтров на данном примере?*

На картинке показано, как из слова извлекаются 2, 3 и 4-граммы. Например, желтые - это триграммы. Желтый фильтр прикладывают ко всем триграммам в слове, а потом с помощью global max-pooling извлекают наиболее сильный сигнал.

Что это значит, если конкретнее?

Каждый символ отображается с помощью эмбеддингов в некоторый вектор. А их последовательности - в конкатенации эмбеддингов.  
Например, "abs" $\to [v_a; v_b; v_s] \in \mathbb{R}^{3 d}$, где $d$ - размерность эмбеддинга. Желтый фильтр $f_k$ имеет такую же размерность $3d$.  
Его прикладывание - это скалярное произведение $\left([v_a; v_b; v_s] \odot f_k \right) \in \mathbb R$ (один из желтых квадратиков в feature map для данного фильтра).

Max-pooling выбирает $max_i \left( [v_{i-1}; v_{i}; v_{i+1}] \odot f_k \right)$, где $i$ пробегается по всем индексам слова от 1 до $|w| - 1$ (либо по большему диапазону, если есть padding'и).   
Этот максимум соответствует той триграмме, которая наиболее близка к фильтру по косинусному расстоянию.

В результате в векторе после max-pooling'а закодирована информация о том, какие из n-грамм встретились в слове: если встретилась близкая к нашему $f_k$ триграмма, то в $k$-той позиции вектора будет стоять большое значение, иначе - маленькое.

А учим мы как раз фильтры. То есть сеть должна научиться определять, какие из n-грамм значимы, а какие - нет.

### Классификация слов

Будем учиться предсказывать, является ли слово фамилией.

Скачаем данные.

In [0]:
!wget -qq -O surnames.txt https://share.abbyy.com/index.php/s/mt5r9vEZo70sfIS/download

In [0]:
from sklearn.model_selection import train_test_split

with open('surnames.txt') as f:
    lines = f.readlines()
    data = [line.strip().split('\t')[0] for line in lines]
    labels = [int(line.strip().split('\t')[1]) for line in lines]
    del lines
    
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

Данные, как видно, грязноваты:

In [0]:
list(zip(X_train, y_train))[:10]

Начнем с бейзлайна - логистической регрессии на n-граммах символов.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3), lowercase=False)

Как всегда, сконвертируем их для начала:

In [0]:
from collections import Counter 
    
def find_max_len(counter, threshold):
    sum_count = sum(counter.values())
    cum_count = 0
    for i in range(max(counter)):
        cum_count += counter[i]
        if cum_count > sum_count * threshold:
            return i
    return max(counter)

word_len_counter = Counter()
for word in X_train:
    word_len_counter[len(word)] += 1
    
threshold = 0.99
MAX_WORD_LEN = find_max_len(word_len_counter, threshold)

print('Max word len for {:.0%} of words is {}'.format(threshold, MAX_WORD_LEN))

In [0]:
chars = set()
for word in X_train:
    chars.update(word)

char_index = {c : i + 1 for i, c in enumerate(chars)}
char_index[''] = 0

def get_char_index(char, char_index):
    return char_index[char] if char in char_index else len(char_index)
  
print(char_index)

In [0]:
def convert_data(data, max_word_len, char_index):
    X = np.zeros((len(data), max_word_len))
    for i, word in enumerate(data):
        word = word[-max_word_len:]
        X[i, :len(word)] = [get_char_index(symb, char_index) for symb in word]
        
    return LongTensor(X)
  
X_train = convert_data(X_train, MAX_WORD_LEN, char_index)
X_test = convert_data(X_test, MAX_WORD_LEN, char_index)

y_train = FloatTensor(y_train)
y_test = FloatTensor(y_test)

In [0]:
def iterate_batches(dataset, batch_size):
    X, y = dataset
    num_samples = X.shape[0]

    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        
        batch_idx = indices[start:end]
        
        yield Variable(X[batch_idx, ]), Variable(y[batch_idx, ])

Теперь построим свёрточную модель.

Типичным является блок:
```python
nn.Conv*d(in_channels=N, out_channels=M, kernel_size=K1, padding=0)
F.relu
nn.MaxPool*d(kernel_size=K2)
```

Пусть она будет строить триграммы - то есть применять фильтры на 3 символа.

Какие нам нужны размерности?

In [0]:
class ConvClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, filters_count):
        super().__init__()
        
        <init layers>
        
    def forward(self, inp):
        '''
        inp.size() = (batch_size, max_word_len)
        out.size() = (batch_size,)
        '''
        <apply layers>

В данной задаче несбалансированные классы, поэтому хочется мерять $F_1$-меру.

Напомню:

![precision-recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png =x600)  
From [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall).

$$\text{precision} = \frac{tp}{tp + fp}.$$
$$\text{recall} = \frac{tp}{tp + fn}.$$
$$\text{F}_1 = 2\frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}.$$

In [0]:
model = ConvClassifier(len(char_index) + 1, 24, 128).cuda()

X_batch, y_batch = next(iterate_batches((X_train, y_train), 32))

<calculate precision, recall and F1-score>

In [0]:
import math
import time

def do_epoch(model, criterion, data, batch_size, optimizer=None):
    epoch_loss = 0
    epoch_tp = 0
    epoch_tpfn = 0
    epoch_tpfp = 0
    
    model.train(not optimizer is None)
    
    batchs_count = math.ceil(data[0].shape[0] / batch_size)
    
    for i, (X_batch, y_batch) in enumerate(iterate_batches(data, batch_size)):
        logits = model(X_batch)
        
        loss = criterion(logits, y_batch)
        epoch_loss += loss.data[0]
        
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        <calculate precision, recall and F1-score for batch>
      
        print('\r[{} / {}]: Loss = {:.5f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'.format(
              i, batchs_count, loss.data[0], precision, recall, f1), end='')
        
    <calculate precision, recall and F1-score for epoch>
        
    return epoch_loss / batchs_count, recall, precision, f1

def fit(model, criterion, optimizer, train_data, epochs_count=1, 
        batch_size=32, val_data=None, val_batch_size=None):
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        start_time = time.time()
        train_loss, train_recall, train_precision, train_f1 = \
            do_epoch(model, criterion, train_data, batch_size, optimizer)
        
        output_info = '\rEpoch {} / {}, Epoch Time = {:.2f}s: Train Loss = {:.5f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'
        if not val_data is None:
            val_loss, val_recall, val_precision, val_f1 = \
                do_epoch(model, criterion, train_data, batch_size, None)
            
            epoch_time = time.time() - start_time
            output_info += ', Val Loss = {:.5f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'
            print(output_info.format(epoch+1, epochs_count, epoch_time, 
                                     train_loss, train_recall, train_precision, train_f1,
                                     val_loss, val_recall, val_precision, val_f1))
        else:
            epoch_time = time.time() - start_time
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss))

In [0]:
model = ConvClassifier(len(char_index) + 1, 24, 256).cuda()

criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad])

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=100, 
    batch_size=512, val_data=(X_test, y_test), val_batch_size=1024)

**Задание** Различают Narrow и Wide свёртки - по сути, добавляется ли нулевой паддинг или нет. Для текстов эта разница выглядит так:  
![narrow_vs_wide](https://image.ibb.co/eqGZaS/2018_03_28_11_23_17.png)
From Neural Network Methods in Natural Language Processing.  
Слева - паддинг отсутствует, справа - есть. Попробуйте добавить паддинг и посмотреть, что получится. Потенциально он поможет выучить хорошие префиксы слова.

--- 

**Задание** Сравните качество и скорость работы с character-level LSTM (типа того, что был на третьем занятии).

### Визуализация полученных свёрток

Мы обучили набор свёрток и эмбеддинги. Давайте посмотрим, на какие именно символы загораются свёртки.

In [0]:
filters = next(model.conv.parameters())
embeddings = next(model.embedding.parameters())

Рассмотрим только маленькие буквы:

In [0]:
def get_range(first_symb, last_symb):
    return [chr(c) for c in range(ord(first_symb), ord(last_symb) + 1)]
  
russian_letters = [''] + get_range('а', 'я')
russian_letters_idx = [char_index[letter] for letter in russian_letters]

Эмбеддинг триграммы - это просто конкатенация эмбеддингов её символов:

In [0]:
suffix = 'сев'

suffix_embedding = torch.cat([embeddings[char_index[letter]] for letter in suffix] + 
                             [embeddings[char_index['']] for i in range(3 - len(suffix))])

Посчитайте, какой из фильтров сильнее всего реагирует на триграмму.

In [0]:
<Find the most similar filter>

А теперь - наоборот, подсчитаем, как найденный фильтр реагирует на все символы из `russian_letters`.

Нужно построить матрицу `sim` размера `3 x len(russian_letters)`, в каждом элементе которой будет записано, насколько сильно данный элемент фильтра реагирует на данный символ.

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

sim = ...

fig = plt.figure(figsize=(30, 5))
ax = fig.add_subplot(111)
cax = ax.matshow(sim, cmap='bone')
fig.colorbar(cax)

ax.set_xticklabels([''] + russian_letters)
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

plt.show()

## Attention and Convolutions

Напомню, что attention работает так: пусть у нас есть набор скрытых состояний $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - например, представлений слов из исходного языка, полученных с помощью энкодера. И есть некоторое текущее скрытое состояние $\mathbf{h}_i$ - скажем, представление, используемое для предсказания слова на нужном нам языке.

Тогда с помощью аттеншена мы можем получить взвешенное представление контекста $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - вектор $c_i$:
$$
\begin{align}\begin{split}
\mathbf{c}_i &= \sum\limits_j a_{ij}\mathbf{s}_j\\
\mathbf{a}_{ij} &= \text{softmax}(f_{att}(\mathbf{h}_i, \mathbf{s}_j))
\end{split}\end{align}
$$

$f_{att}$ - функция, которая говорит, насколько хорошо $\mathbf{h}_i$ и $\mathbf{s}_j$ подходят друг другу.

Самые популярные её варианты:
- Additive attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{v}_a{}^\top \text{tanh}(\mathbf{W}_a[\mathbf{h}_i; \mathbf{s}_j])$$
- Dot attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{s}_j$$
- Multiplicative attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{W}_a \mathbf{s}_j$$

Есть ещё одна вариация на тему - self-attention. Это когда у нас нет $\mathbf{h}_i$ - только $\mathbf{s}_j$-тые, т.е. вектора-представления исходной последовательности. Такое может очень естественно возникнуть практически в любой задаче - например, в классификации текстов.

Additive self-attention можно записать как $f_{att}(\mathbf{s}_i) = \mathbf{v}_a{}^\top \text{tanh}(\mathbf{W}_a \mathbf{s}_i)$. Тогда по последовательности $\mathbf{s}_1, \ldots, \mathbf{s}_m = \mathbf{S}$ вычисляется единственный вектор:
$$
\begin{align}\begin{split}
\mathbf{a} &= \text{softmax}(\mathbf{v}_a \text{tanh}(\mathbf{W}_a \mathbf{S}^\top))\\
\mathbf{c} & = \mathbf{S} \mathbf{a}^\top
\end{split}\end{align}
$$

При чём здесь были упомянуты свёртки? В основном при том, что они, вообще говоря, действуют похожим образом. 

Давайте посмотрим на операцию $\mathbf{v}_a \text{tanh}(\mathbf{W}_a \mathbf{S}^\top)$ как на извращенный вариант свертки с размером фильтра 1. Для каждого вектора $\mathbf{s_j}$ строится некоторая оценка того, насколько важно данное состояние.

Кроме того, кроме max-pooling'а существует ещё и avg-pooling - когда берется не максимальный сигнал, полученный сверткой, а средний. В какой-то степени $\mathbf{c} = \mathbf{S} \mathbf{a}^\top$ - тоже усреднение. Правда, усредняются исходные вектора, а не получающиеся оценки.

В таком self-attention мы выучиваем только один аналог фильтра сверточной сети. Но в сверточных сетях же много фильтров. Давайте тогда учить сразу несколько вариантов attention'а:
$$
\begin{align}\begin{split}
\mathbf{A} &= \text{softmax}(\mathbf{V}_a \text{tanh}(\mathbf{W}_a \mathbf{H}^\top))\\
\mathbf{C} & = \mathbf{A} \mathbf{H}
\end{split}\end{align}
$$

Утверждается, что случайной инициализации вполне достаточно, чтобы attention начал отлавливать разные характеристики из последовательности - такие лексическое и грамматическое значение слов. Но это и вполне естественно, если посмотреть, что свертки уже много лет так учат.

## Attention Is All You Need

В середине прошлого года вышла статья [Attention Is All You Need](https://arxiv.org/abs/1706.03762), которая опирается на эти идеи.

Она описывает Transformer - архитектуру полносвязной сети, которая делает то, что раньше все делали с помощью рекуррентных сетей. Хороший обзор был на хабре: [Transformer — новая архитектура нейросетей для работы с последовательностями](https://habrahabr.ru/post/341240/).

К задаче машинного перевода она применима так:  
![transformer](https://hsto.org/webt/59/f0/44/59f04410c0e56192990801.png =x600)

Из интересного - multi-head attention, который является аналогом того, что мы рассматривали в предыдущем разделе:  
![multi-head attn](https://hsto.org/webt/59/f0/44/59f0440f1109b864893781.png)

Сам attention выглядит так: 
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^\top}{\sqrt{d_k}} \right) V,$$

Выглядит страшно, но в реальности передают в качестве $Q, K, V$ одну и ту же последовательность скрытых состояний - $\mathbf{S}$ из предыдущего раздела.

$Q K^\top$ - это как dot-attention.

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) W^o$$ 
$$\text{head}_i = \text{Attention}(Q W^Q_i, K W^K_i, V W^V_i).$$

На pytorch это выглядит так (утащено из [attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch) и [seq2seq.pytorch](https://github.com/eladhoffer/seq2seq.pytorch)):

In [0]:
class ScaledDotProductAttention(nn.Module):
    """
    Scaled Dot-Product Attention
    """

    def __init__(self, dropout=0, causal=False):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v):
        b_q, t_q, dim_q = q.size()
        b_k, t_k, dim_k = k.size()
        b_v, t_v, dim_v = v.size()
        qk = torch.bmm(q, k.transpose(1, 2))  # b x t_q x t_k
        qk.div_(dim_k ** 0.5)
        sm_qk = F.softmax(qk, dim=2)
        sm_qk = self.dropout(sm_qk)
        return torch.bmm(sm_qk, v), sm_qk  # b x t_q x dim_v


class MultiHeadAttention(nn.Module):
    """
    Scaled Dot-Product Attention
    """

    def __init__(self, input_size, output_size, num_heads, dropout=0):
        super(MultiHeadAttention, self).__init__()
        assert(input_size % num_heads == 0)
        self.input_size = input_size
        self.output_size = output_size
        self.num_heads = num_heads
        
        self.linear_q = nn.Linear(input_size, input_size)
        self.linear_k = nn.Linear(input_size, input_size)
        self.linear_v = nn.Linear(input_size, input_size)
        self.linear_out = nn.Linear(input_size, output_size)
        
        self.sdp_attention = ScaledDotProductAttention(dropout=dropout)

    def forward(self, q, k, v):
        b_q, t_q, dim_q = q.size()
        b_k, t_k, dim_k = k.size()
        b_v, t_v, dim_v = v.size()
        
        qw = self.linear_q(q)
        kw = self.linear_k(k)
        vw = self.linear_v(v)
        
        qw = qw.chunk(self.num_heads, 2)
        kw = kw.chunk(self.num_heads, 2)
        vw = vw.chunk(self.num_heads, 2)
        
        output = []
        attention_scores = []
        for i in range(self.num_heads):
            out_h, score = self.sdp_attention(qw[i], kw[i], vw[i])
            output.append(out_h)
            attention_scores.append(score)

        output = torch.cat(output, 2)

        return self.linear_out(output), attention_scores

Чтобы всё это заработало, нужно смотреть не только на эмбеддинги слов, но и на их позиции в тексте. Предлагается использовать позиционное кодирование вида:
$$\text{PE}_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{model}})$$
$$\text{PE}_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{model}})$$

Хотя можно и просто учить эмбеддинги.

Их можно предподсчитать:

In [0]:
def position_encoding_init(n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos_vec) for j in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    return torch.from_numpy(position_enc).type(FloatTensor)

Энкодер выглядит просто:

In [0]:
class EncoderBlock(nn.Module):

    def __init__(self, hidden_size=512, num_heads=8, inner_linear=1024, layer_norm=True, dropout=0):

        super(EncoderBlock, self).__init__()
        if layer_norm:
            self.lnorm1 = LayerNorm1d(hidden_size)
            self.lnorm2 = LayerNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.attention = MultiHeadAttention(hidden_size, hidden_size, num_heads, dropout=dropout)
        self.fc = nn.Sequential(nn.Linear(hidden_size, inner_linear),
                                nn.ReLU(inplace=True),
                                nn.Dropout(dropout),
                                nn.Linear(inner_linear, hidden_size))

    def forward(self, x):
        res = x
        x, _ = self.attention(x, x, x)
        x = self.dropout(x).add_(res)
        x = self.lnorm1(x) if hasattr(self, 'lnorm1') else x
        res = x
        x = self.fc(x)
        x = self.dropout(x).add_(res)
        x = self.lnorm2(x) if hasattr(self, 'lnorm2') else x
        return x

Декодер отличается только необходимостью добавить маскинг - он должен смотреть только на предыдущие сгенерированные состояния, но не на следующие, и attention'ом на выход энкодера.